<a href="https://colab.research.google.com/github/fxr1115/Learning/blob/main/Python3-Algorithm/array-string/3_string.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from typing import List

### 字符串

- 字符串的基本操作对象通常是字符串整体或者其子串
- 字符串操作比其他数据类型更复杂
- Python可以使用`==`来比较两个字符串
- Python中，字符串**不可变**——进行字符串的连接操作会带来一些问题，哪怕只是修改其中一个字符，也需要创建一个新的字符串

### 题目1 最长公共前缀
编写一个函数来查找字符串数组中的最长公共前缀，如果不存在公共前缀，返回空字符串

**疏忽**：没有考虑到长度问题，索引会超过一些`str`
- `str.join(iterable)`拼接字符串

In [4]:
def longestCommonPrefix(strs: List[str]) -> str:
    con = []
    fir = strs[0]
    for i in range(len(fir)):
        for str in strs[1:]:
            if len(str) < i:
                return ''.join(con)
            else:
                if fir[i] != str[i]:
                    return ''.join(con)
        con.append(fir[i])
    return ''.join(con)

longestCommonPrefix(["flower","flow","flight"])

'fl'

**优化**：
- 多次对字符串进行长度检查，重复执行，可以*提前做检查*
- 先用con保存再拼接，多余，*直接使用字符串*处理更高效
- 嵌套循环中，每次都要索引访问`fir[i]`和`str[i]`，可以通过*短路优化*

**注**：
- `startswith()`是Python字符串的内置方法，判断是否以指定前缀开头

In [6]:
def longestCommonPrefix(strs: List[str]) -> str:
    if not strs:
        return ''
    # 找到最短字符串长度，初始化为第一个字符串的前min_len个字符
    min_len = min(len(str) for str in strs)
    prefix = strs[0][:min_len]

    for str in strs:
        while not str.startswith(prefix):
            prefix = prefix[:-1]
            if not prefix:
                return ''
    return prefix

longestCommonPrefix(["flower","flow","flight"])

'fl'

使用**字典排序**，优化公共前缀的查找
- 通过公共前缀的查找，则公共前缀一定是`strs[0]`和`strs[-1]`的公共前缀

In [9]:
def longestCommonPrefix(strs: List[str]) -> str:
    str1 = ''
    strs.sort()

    for i in range(min(len(str) for str in strs)):
        if strs[0][i] == strs[-1][i]:
            str1 += strs[0][i]
        else:
            break
    return str1

longestCommonPrefix(["flower","flow","flight"])

'fl'

### 题目2 最长回文子串
给一个字符串，找到字符串中最长的 回文子串（正反读都是一样的内容）

In [11]:
def longestPalindrome(s: str) -> str:
    n = len(s)
    result = s[0]
    for i in range(2, n + 1):
        for j in range(n - i + 1):
            str = s[j: j + i]
            if str == str[::-1]:
                result = str
    return result

longestPalindrome("babad")

'aba'

**代码改进**：（子串的重复检查，复杂度较高）
- 生成并检查*所有可能*的子串，即使有些子串已经被验证过了，也会再次检查
- 每次通过*切片和反转*检查回文，增加重复计算
- 整体复杂度为**$O(n^3)$**，暴力枚举方式在较长字符串时性能会很差<外层循环、内层循环、切片反转>

注：
- `s[a, b]`，切片是左闭右开的区间，取不到`b`

**优化**：**中心扩展法**
- 利用回文的对称性，以每个字符或字符间隙为中心，向两侧扩展找到最长回文子串，复杂度为O(n^2)
- 避免了生成和遍历所有子串，*只需检查**$O(n)$**个中心点*，而每个中心扩张的复杂度为$O(n)$，所以**总复杂度为$O(n^2)$**
- *不再需要*字符串切片和反转操作，直接通过指针扩展判断回文

In [16]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        n = len(s)
        if n <= 1:
            return s[0]

        start, max_len = 0, 0
        for i in range(n):
            len1 = self.expand_around_center(s, i, i)     # 奇数长回文
            len2 = self.expand_around_center(s, i, i + 1)  # 偶数长回文
            curr_len = max(len1, len2)
            # 更新回文起始位置和长度
            if curr_len > max_len:
                max_len = curr_len
                start = i - (curr_len - 1) //2
        return s[start: start + max_len]

    def expand_around_center(self, s, left, right):
        # 从中心扩展，找到最长回文
        while left >= 0 and right < len(s) and s[left] == s[right]:
            left -= 1
            right += 1
        return right - left - 1

Solution().longestPalindrome("babad")

'bab'

#### **动态规划**
- 通过子串是否为回文来避免重复计算，时间复杂度$O(n^2)$
- 利用**二维数组**记录子串是否为回文

- 核心思想
    - 回文的性质：
        - 一个子串`s[i:, j+1]`是回文，当且仅当
            - `s[i] = s[j]`（子串两端字符相等）
            - `s[i+1: j]`是回文（去掉首尾字符后的子串是回文）
    - 状态定义：
        - 定义一个二维布尔数组`dp`，其中`dp[i][j]`表示字符串`s[i: j+1]`是否为回文
    - 状态转移方程：
        - 如果`s[i]==s[j]`，则：
            - 当子串长度为1或2时，`dp[i][j] = True`，即单个字符或两个相等的字符是回文
            - 当子串长度大于2时，`dp[i][j] = dp[i+1][j-1]`，即去掉收尾后的子串也需是回文
    - 初始化：
        - 所有单个字符是回文，`dp[i][j] = True`
    - 结果：
        - 通过遍历`dp`找到最长的回文子串


In [19]:
def longestPalindrome(s: str) -> str:
    n = len(s)
    if n <= 1:
        return s[0]

    # 生成一个n*n矩阵
    dp = [[False] * n for _ in range(n)]
    start, max_len = 0, 1

    for i in range(n):
        dp[i][i] = True

    for length in range(2, n + 1):
        for i in range(n - length + 1):  # 遍历起始位置
            j = i + length - 1     # 结束位置

            if s[i] == s[j]:
                if length == 2:
                    dp[i][j] = True
                elif s[i + 1] == s[j - 1]:
                    dp[i][j] = True

            if dp[i][j] and length > max_len:
                start = i
                max_len = length
    return s[start: start + max_len]

longestPalindrome("babad")

'bab'

**Manacher算法**：（比较复杂，有时间再看）
- 是一个线性时间算法O(n)，通过在字符串中插入特殊字符来避免奇偶回文的判断，并利用已有的回文信息来减少计算
- 核心思想：
    - 奇偶回文统一处理：在每个字符之间插入特殊字符`#`，将奇数长度和偶数长度的回文统一处理，如`abc`为`#a#b#c#`
    - 记录半径数组：定义一个数组`p`，`p[i]`表示以位置`i`为中心的回文半径（不包括中心点）
    - 中心扩展优化：用变量`center`表示当前回文的中心，`right`表示这个回文的右边界
        - 如果`i`位于`right`的范围内，可以利用对称性直接得出部分回文信息，不需重新计算
    - 结果还原：通过数组`p`找到最大回文的中心和半径，从而提取出原始文字中的最长回文

### 题目3 翻转字符串中的单词
给一个字符串，反转字符串中的单词顺序
- （返回单词顺序颠倒且单词之间用单个空格连接的结果字符串，并且不存在前导空格和尾随空格）

- **`.split()`**，即使用单个空格作为分隔符，多个连续的空格会被视为分隔符之间的空内容，得到空字符串`''`

In [20]:
def reverseWords(s: str) -> str:
    s = s.strip()
    s_list = s.split(' ')
    s_list1 = []
    for str in s_list:
        if str:
            str.strip()
            s_list1.append(str)
    s_list2 = s_list1[::-1]
    return ' '.join(s_list2)

reverseWords("the sky is blue")

'blue is sky the'

**优化**：
- *避免不必要的遍历*：对`s_list`遍历两次（分隔过滤空字符串，反转列表），可以直接使用`split()`，会自动忽略多余的空格
- 不需要显式调用`strip()`，当**`split()`不传参**时，会自动移除字符串两端的空白字符
- *简化反转*：直接在分割后调用`[::-1]`反转列表，无需显示创建`s_list1`和`s_list2`

In [21]:
def reverseWords(s: str) -> str:
    return ' '.join(s.split()[::-1])

reverseWords("the sky is blue")

'blue is sky the'

### **字符串匹配算法**：**KMP**(Knuth-Morris-Pratt)


- 一种改进的字符串匹配算法，核心是利用匹配失败后的信息，尽量减少模式串与主串的匹配次数以达到快速匹配的目的，时间复杂度是O(m+n)
- 思想：（B站）
    - 模式串是主要，分析模式串就可以匹配任意的主串：每次开始比较的位置，就是当前最大公共前后缀长度加一
- **还没明白**

### 题目4 找出字符串中第一个匹配项的下标
在`haystack`字符串中找到`needle`字符串的第一个匹配项的下标（从0开始），如果没有匹配的，返回`-1`


In [22]:
def strStr(haystack: str, needle: str) -> int:
    m, n = len(haystack), len(needle)
    for i in range(m - 1 + 1):
        if needle == haystack[i: i + n]:
            return i
        return -1

strStr("sadbutsad", "sad")

0